In [4]:
from os.path import join, isdir, exists
from os import listdir
import pickle
from collections import Counter
import pandas as pd
import re

In [5]:
nsyn_dir = '../mutsig_out/nsyn/out10212020'
syn_dir = '../mutsig_out/syn/out10212020'
org_dir = '../org_cov_maf'

outdir = '../proc_09152020/dovgene'

In [29]:
def get_dictionary(org):
    f = join(outdir,org+'_102120.tsv' )
    
    dmut = {}
    
    with open(f,'r') as orgf:
        for lines in orgf:
            line_split = lines.split('\t')
            if len(line_split) > 2:
                gene,mut,vc,ovpgene,cons = lines.split('\t')
                cons = cons.strip('\n')
                cons = re.findall(r"'(.*?)'", cons)

                if gene not in dmut:
                    dmut[gene] = {}
                if mut not in dmut[gene]:
                    dmut[gene][mut] = {}
                    dmut[gene][mut]['ovp'] = {}
                    dmut[gene][mut]['vc'] = vc
                if ovpgene not in dmut[gene][mut]['ovp']:
                    blacklist = ['downstream_gene_variant','upstream_gene_variant']
                    cons = [v for v in cons if v not in blacklist]

                    if not cons:
                        dmut[gene][mut] = {}
                        dmut[gene][mut]['ovp'] = {}
                        dmut[gene][mut]['vc'] = None
                    else:
                        dmut[gene][mut]['ovp'][ovpgene] = str(cons)
            else:
                gene,mut = lines.split('\t')
                if gene not in dmut:
                    dmut[gene] = {}
                if mut not in dmut[gene]:
                    dmut[gene][mut] = {}
                    dmut[gene][mut]['ovp'] = {}
                    dmut[gene][mut]['vc'] = None
                

    return dmut

In [30]:
def get_area(lymf,gene):
    larea =[]
    for muts in lymf[gene]:
        larea.append(lymf[gene][muts]['vc'])
            
    return dict(Counter(larea))

In [13]:
def get_ovp_gene(lymf,gene):
    lcg = []
    for muts in lymf[gene]:
        for ovpgene in lymf[gene][muts]['ovp']:
            if ovpgene not in lcg:
                lcg.append(ovpgene)
    
    return lcg

In [14]:
def get_ovp_area(lymf,gene):
    lovp = []
    for muts in lymf[gene]:
        for ovpgene in lymf[gene][muts]['ovp']:
            lovp.append(lymf[gene][muts]['ovp'][ovpgene])
            
    return dict(Counter(lovp))

In [31]:
# this little function get the significant gene file path by organ name
lorg = listdir(org_dir)
for organ in lorg:
    if not organ.startswith('.') and not organ == 'SoftTissue':
        orgf = join(outdir,organ+'_102120.tsv' )
        # get gene list for query
        organ_dir = join(syn_dir,organ)
        for f in listdir(organ_dir):
            if f.endswith('sig_genes.txt'):
                f_dir = join(organ_dir,f)
                df_org = pd.read_csv(f_dir,sep = '\t', index_col=None)
                
                dictmut = get_dictionary(organ)
                
                for index, row in df_org.iterrows():
                    gene = df_org.loc[index,'gene']
                    try:
                        narea = get_area(dictmut,gene)
                        lovpgene = get_ovp_gene(dictmut,gene)

                        if narea.keys() == [None]:
                            df_org.loc[index,'ovp_area'] = 0
                        else:
                            del narea[None]
                            df_org.loc[index,'ovp_area'] = str(narea)
                        df_org.loc[index,'ovp_gene'] = str(lovpgene)

                        lovparea = []
                        for g in lovpgene:
                            ovparea = get_ovp_area(dictmut,gene)
                            lovparea.append(ovparea)

                        df_org.loc[index,'ovp_gene_area'] = str(lovparea)
                    except KeyError:
                        pass
                        
                cols = ['gene', 'n_nonsilent', 'n_silent','n_noncoding', 'nnei', 'x','X','p','q', 'ovp_area', 'ovp_gene', 'ovp_gene_area']

                df_org = df_org[cols]
                df_org = df_org[df_org['ovp_area'].notnull()]
                
#                 write to output
                outpsyn = 'ovpgene_anlyze_1029.csv'
                if not exists(outpsyn):
                    with open(outpsyn, 'wb') as outfsyn:
                        outfsyn.write(organ+'\n')
                else: 
                    with open(outpsyn, 'a') as outfsyn:
                        outfsyn.write(organ+'\n')
                df_org.to_csv(outpsyn, mode='a', header=True, sep= ',')

In [67]:
outf='../mutsig_out/anlyze/lymph_syn_res.cvs'
df_org.to_csv(outf, mode='wb', header=True, sep= '\t', index = False)